# AED - Análise Exploratória de Dados O₃ (Ozônio) - Diário

## 1. Carregamento e Preparação dos Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("la_o3_daily.csv", parse_dates=["timestamp"])
df = df.set_index("timestamp").sort_index()

if df.index.tz is not None:
    df.index = df.index.tz_localize(None)

print(df.head())
print(df.tail())
print(df.info())

## 2. Informações Básicas da Série

In [ ]:
print("Período de cobertura:", df.index.min().date(), "→", df.index.max().date())
print("Total de pontos:", len(df))

## 3. Estatísticas Descritivas

In [ ]:
print(df['o3_ug_m3'].describe())

## 4. Qualidade dos Dados

In [ ]:
print("Nulos por coluna:\n", df.isna().sum())
print("Duplicatas no índice:", df.index.duplicated().sum())

## 5. Análise de Intervalos e Datas Faltantes

In [ ]:
# Intervalos observados
intervalos = df.index.to_series().diff().dropna()
print("Intervalos mais comuns:\n", intervalos.value_counts().head())

# Construir grade diária completa e checar datas faltantes
full_range = pd.date_range(df.index.min(), df.index.max(), freq='D')
missing_dates = full_range.difference(df.index)
print("Datas faltantes:", list(missing_dates))

# Versão reindexada (opcional) para análises que exigem regularidade
df_daily = df.reindex(full_range)
df_daily.rename_axis('timestamp', inplace=True)

## 6. Visualizações Básicas

In [ ]:
plt.figure(figsize=(11,4))
plt.plot(df.index, df['o3_ug_m3'], marker='o')
plt.title('O₃ diário')
plt.xlabel('Data'); plt.ylabel('O₃ (µg/m³)')
plt.tight_layout(); plt.show()

# Histograma
plt.figure(figsize=(6,4))
df['o3_ug_m3'].plot(kind='hist', bins=12, title='Distribuição O₃ (diário)')
plt.xlabel('O₃ (µg/m³)')
plt.tight_layout(); plt.show()

# Boxplots por dia da semana
by_wd = df.copy()
by_wd['weekday'] = by_wd.index.day_name()
order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
by_wd['weekday'] = pd.Categorical(by_wd['weekday'], categories=order, ordered=True)

plt.figure(figsize=(10,4))
by_wd.boxplot(column='o3_ug_m3', by='weekday', grid=False)
plt.suptitle('O₃ por dia da semana'); plt.title('')
plt.xlabel('Dia da semana'); plt.ylabel('O₃ (µg/m³)')
plt.tight_layout(); plt.show()

# Médias por dia da semana (barra)
plt.figure(figsize=(8,3.5))
by_wd.groupby('weekday')['o3_ug_m3'].mean().plot(kind='bar')
plt.title('Média de O₃ por dia da semana')
plt.ylabel('µg/m³'); plt.xlabel('Dia da semana')
plt.tight_layout(); plt.show()

## 7. Rolling Statistics

In [ ]:
roll = df['o3_ug_m3'].rolling(window=7, min_periods=3)

plt.figure(figsize=(11,4))
plt.plot(df.index, df['o3_ug_m3'], label='Série', alpha=0.7)
plt.plot(roll.mean(), label='Média móvel (7d)')
plt.plot(roll.std(), label='Desvio padrão móvel (7d)')
plt.legend(); plt.title('Rolling stats (7 dias)')
plt.tight_layout(); plt.show()

## 8. Decomposição STL (Seasonal-Trend decomposition using Loess)

In [ ]:
from statsmodels.tsa.seasonal import STL

# Usar a série reindexada/preenchida para STL caso haja gaps
s = df_daily['o3_ug_m3']
s_interp = s.interpolate(limit_direction='both')

stl = STL(s_interp, period=7, robust=True)
res = stl.fit()
res.plot()
plt.suptitle('STL (period=7, robust=True)')
plt.show()

# Força sazonal aproximada (Hyndman): 1 - Var(remainder)/Var(seasonal+remainder)
seasonal_var = np.var(res.seasonal)
remainder_var = np.var(res.resid)
seasonality_strength = 1 - (remainder_var / (seasonal_var + remainder_var))
print(f"Força sazonal (aprox): {seasonality_strength:.3f}")

## 9. Testes de Estacionariedade

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

y = df['o3_ug_m3'].astype(float)

# Dickey-Fuller (H0: não-estacionária)
adf_stat, adf_p, *_ = adfuller(y.dropna(), autolag='AIC')
print(f"ADF: stat={adf_stat:.3f} | p={adf_p:.4f}  -> p<0.05 sugere estacionária")

# KPSS (H0: estacionária em nível). Pode falhar com amostras pequenas; usar try/except.
try:
    kpss_stat, kpss_p, *_ = kpss(y.dropna(), regression='c', nlags='auto')
    print(f"KPSS: stat={kpss_stat:.3f} | p={kpss_p:.4f}  -> p<0.05 sugere não-estacionária")
except Exception as e:
    print("KPSS não pôde ser calculado:", e)

## 10. ACF e PACF

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(y, lags=20)
plt.title('ACF (até 20 lags)')
plt.show()

plot_pacf(y, lags=20, method='ywm')
plt.title('PACF (até 20 lags)')
plt.show()

## 11. Detecção de Outliers

In [ ]:
# Z-score simples
z = (y - y.mean()) / y.std(ddof=0)
outliers = y[np.abs(z) > 3]
print("Outliers (|z|>3):")
print(outliers)

# Diferenças absolutas dia-a-dia (para possíveis quebras)
diff1 = y.diff().abs()
print("Top 5 maiores variações diárias:\n", diff1.sort_values(ascending=False).head())

## 12. STL com período alternativo (30 dias)

In [ ]:
stl30 = STL(s_interp, period=30, robust=True).fit()
stl30.plot()
plt.suptitle('STL (period=30) – inspeção')
plt.show()

## 13. Resumo Consolidado

In [ ]:
print("Cobertura:", df.index.min().date(), "→", df.index.max().date(), "| pontos:", len(df))
print(df['o3_ug_m3'].describe())
print("Nulos:", df.isna().sum().to_dict(), "| Duplicatas no índice:", df.index.duplicated().sum())

## 14. Visualizações Adicionais

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df.index, df['o3_ug_m3'], lw=1)
plt.title("O₃ diário (série completa)")
plt.xlabel("Data"); plt.ylabel("µg/m³")
plt.tight_layout(); plt.show()

# Rolling de 7 e 30 dias
roll7 = df['o3_ug_m3'].rolling(7)
roll30 = df['o3_ug_m3'].rolling(30)

plt.figure(figsize=(12,4))
plt.plot(df.index, df['o3_ug_m3'], alpha=.6, label='Série')
plt.plot(roll7.mean(), label='Média móvel 7d')
plt.plot(roll30.mean(), label='Média móvel 30d')
plt.legend(); plt.title("Médias móveis")
plt.tight_layout(); plt.show()

plt.figure(figsize=(12,4))
plt.plot(roll7.std(), label='Desvio padrão 7d')
plt.plot(roll30.std(), label='Desvio padrão 30d')
plt.legend(); plt.title("Heterocedasticidade (rolling std)")
plt.tight_layout(); plt.show()

## 15. Padrões Sazonais (Dia da Semana e Mês)

In [ ]:
tmp = df.copy()
tmp['weekday'] = tmp.index.day_name()
tmp['month'] = tmp.index.month_name()

order_wd = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
tmp['weekday'] = pd.Categorical(tmp['weekday'], categories=order_wd, ordered=True)

plt.figure(figsize=(10,3.6))
tmp.groupby('weekday')['o3_ug_m3'].mean().plot(kind='bar')
plt.title('Média por dia da semana'); plt.ylabel('µg/m³'); plt.tight_layout(); plt.show()

plt.figure(figsize=(10,3.6))
(tmp.groupby(tmp.index.month)['o3_ug_m3'].mean()
 .rename(index={1:'Jan',2:'Fev',3:'Mar',4:'Abr',5:'Mai',6:'Jun',7:'Jul',8:'Ago',9:'Set',10:'Out',11:'Nov',12:'Dez'})
 .plot(kind='bar'))
plt.title('Média por mês'); plt.ylabel('µg/m³'); plt.tight_layout(); plt.show()

## 16. Decomposição STL Multi-nível (Semanal + Anual)

In [ ]:
from statsmodels.tsa.seasonal import STL

y = df['o3_ug_m3'].astype(float)

stl7 = STL(y, period=7, robust=True).fit()
stl365 = STL(y - stl7.seasonal, period=365, robust=True).fit()  # remove semanal e decompõe anual no restante

stl7.plot(); plt.suptitle('STL semanal (p=7)'); plt.show()
stl365.plot(); plt.suptitle('STL anual (p=365) no restante'); plt.show()